In [58]:
import numpy as np
import cv2
from skimage import io, data
import napari
import matplotlib.pyplot as plt
import copy


#img = plt.imread('data/uoedh_culver house letters12659.jpg')


### Configuration

In [59]:


SHOW_FULLSIZE = False
DELAY = 2000                # in milliseconds
INPUT_PATH = 'data/'
OUTPUT_PATH = 'output/'
OUTPUT_THRESHOLD = 64       # Threshold for final output
CROPPING_THRESHOLD = 1      # Threshold used to select the crop outline

fname = 'uoedh_culver house letters12659'


### Helper functions
* **makethumb()** is a quick rescaling function, reducing an image to 10% linear size
* **showbriefly()** shows an image for DELAY milliseconds or until a key is pressed

In [60]:
def makethumb(i):
    return cv2.resize(i,None,fx=0.1,fy=0.1,interpolation=cv2.INTER_CUBIC)

def showbriefly(i):
    WindowName = 'Image Viewer'
    view_window = cv2.namedWindow(WindowName, cv2.WINDOW_NORMAL)
    w = cv2.imshow(WindowName,i)
    # force focus
    cv2.setWindowProperty(WindowName,cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    cv2.setWindowProperty(WindowName,cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_NORMAL)
    # wait 10 seconds
    cv2.waitKey(DELAY)
    cv2.destroyAllWindows()

### Read image using CV2 functions
* Image is loaded as a BGR matrix

In [61]:
img = cv2.imread(filename=INPUT_PATH+fname+'.jpg')

Convert image to greyscale

In [62]:
grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

### Use cv.threshold() to isolate writing
* note that **cv2.threshold()** returns two arguments:
 * **ret** the threshold mask pattern (temporary variable)
 * **thresh** the image with threshold applied
 * Uses the adaptive **OTSU** algorithms for better results on uneven exposures

In [63]:
ret,thresh = cv2.threshold(grey,OUTPUT_THRESHOLD,255,cv2.THRESH_BINARY)

In [64]:
showbriefly(makethumb(thresh))

#### Find the Contours
* Redo the threshold with more extreme settings
* Find the contours of the objects in the image


From: https://stackoverflow.com/questions/13538748/crop-black-edges-with-opencv

In [65]:
_, cropthresh = cv2.threshold(thresh,CROPPING_THRESHOLD,255,cv2.THRESH_BINARY)

contours,hierarchy = cv2.findContours(cropthresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#showbriefly(newimg)

In [66]:
maxc = []
maxa = 0
for c in contours:
    a = cv2.contourArea(c)
    if a>maxa:
        maxc=c
        maxa=a
        print('new maximum: '+str(maxa))
cont = maxc

new maximum: 0.5
new maximum: 1.0
new maximum: 2.0
new maximum: 4.5
new maximum: 8.5
new maximum: 16.5
new maximum: 17.0
new maximum: 18.0
new maximum: 19.0
new maximum: 33.5
new maximum: 15914667.5


#### Find the bounding rectangle of that maximal contour

In [67]:
x,y,w,h = cv2.boundingRect(cont)

print('x,y,x`,y` = ',x,y,x+w,y+h)
print('w,h = ',w,h)

x,y,x`,y` =  136 81 3674 4726
w,h =  3538 4645


#### Show the bounding box in green for confirmation
* Note that the **rectangle()** function modifies the image it references
 * Hence the need to do a **deepcopy()**

In [68]:
boundbox = copy.deepcopy(img)
cv2.rectangle(boundbox,(x,y),(x+w,y+h),(0,255,0),20)
showbriefly(boundbox)

#### Crop the original image using numpy matrix slicing

In [69]:
crop = img[y:y+h,x:x+w]
showbriefly(crop)

#### And now thresholded image

In [70]:
cropocr = thresh[y:y+h,x:x+w]
showbriefly(cropocr)

#showbriefly(makethumb(img))
#plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#plt.show()

In [71]:
cv2.destroyAllWindows()